In [ ]:
!python -m spacy download en_core_web_lg 

2022-07-30 23:47:19.702842: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 9.1 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
!pip install OCTIS
!python -m spacy download en

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-07-30 23:48:31.143366: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence

from octis.models.LDA import LDA
from octis.models.CTM import CTM
from octis.models.NeuralLDA import NeuralLDA
from octis.models.ProdLDA import ProdLDA
from octis.models.NMF import NMF
from octis.models.ETM import ETM

import os
import string
import sys
from octis.preprocessing.preprocessing import Preprocessing

import pandas as pd
import numpy as np
import inspect
import matplotlib.pyplot as plt
import time
import json
from numpy import transpose
from wordcloud import WordCloud
from matplotlib.pyplot import figure
from PIL import Image
#from bertopic import BERTopic

In [ ]:
# Run this to mount the Notebook in your Google Drive account 
from google.colab import drive
#drive.mount('/content/drive')
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
os.chdir(os.path.pardir)

# Initialize preprocessing
preprocessor = Preprocessing(vocabulary=None, max_features=None,
                             remove_punctuation=True, punctuation=string.punctuation,
                             lemmatize=True, stopword_list='english',
                             min_chars=1, min_words_docs=0)
# preprocess
dataset = preprocessor.preprocess_dataset(documents_path=r'/content/drive/My Drive/nasa/OCTIS/titles.csv')

# save the preprocessed dataset
dataset.save('zot')

created vocab
4856


Open our best output

In [ ]:
ctm_output = {}

with open("/content/drive/My Drive/nasa/OCTIS/ctm_1", "r") as fp:
   ctm_output['topics'] = json.load(fp)

with open("/content/drive/My Drive/nasa/OCTIS/ctm_2", "r") as fp:
   ctm_output['topic-document-matrix'] = json.load(fp)

with open("/content/drive/My Drive/nasa/OCTIS/ctm_3", "r") as fp:
   ctm_output['test-topic-document-matrix'] = json.load(fp)

with open("/content/drive/My Drive/nasa/OCTIS/ctm_4", "r") as fp:
   ctm_output['topic-word-matrix'] = json.load(fp)

We use SpaCy to vectorize our word lists. 
'en_core_web_small' performed quite poorly but large did ok

In [ ]:
import spacy

nlp = spacy.load('en_core_web_lg')

Our list of predefined topics, with manual input into some words that are meaningful. One thing that we found was that having too many words here could be very detrimental to the similarity values found between that topic and anything it was compared to, so making sure that each word in a list has relatively same weighting is important. Or, an implementation could be done in the future to weight these words as well.

In [ ]:
p_topics = [
    ["aerosol","air quality"],
    ["weather", "precipitation"],
    ["climate","global warming"],
    ["carbon dioxide","ozone"],
    ["ocean", "marine"],
    ["wildfire", "smoke"],
    ["agricult","farm"],
    ["vegetation"],
    ["wildlife", "animal"],
    ["severe weather","extreme heat","storm","tornado","lightning"],
    ["drought","runoff","water resource"],
    ["water quality","algal","coral bleach","eutrophication", "nutrient pollution"],
    ["validation", "verification"],
    ["algorithm", "model", "neural network"],
    ["dust storm","sandstorm"],
    ["flood","inundation"],
    ["health", "uv"],
    ["hurricane","cyclone","typhoon"],
    ["volcan","lava","tephra","tuff","pyroclastic"],
    ["landslide","rockfall", "avalanche", "mudslide", "mudflow", "debris"],
    ["calibration", "gauge"],
    ["earthquake","seismic","tremor"]
]

In [ ]:
ctm_output['topics'][6]

['sub', 'east', 'omi', 'asia', 'tropospheric', 'i', 'x', 'term', 'lt', 'long']

some testing cells

In [ ]:
vals = [0] * 40
for topic in range(40):
  temp_val = []
  indices = np.flip(np.argsort(ctm_output['topic-word-matrix'][topic])[-10:], 0).tolist()
  for i in indices:
    temp_val.append(max(ctm_output['topic-word-matrix'][topic][i] - 0.15, 0))
  vals[topic] = temp_val


In [ ]:

ctm_topic = 7
topic_totals = []
for t in p_topics:

  predefined_len = len(t)
  predefined_spacy = [nlp(x) for x in t]
  ctm_spacy = [nlp(x) for x in ctm_output['topics'][ctm_topic]]
  val_list = vals[ctm_topic]
  val_total = np.sum(val_list)

  total = 0
  for y in range(8):
    sim = []
    multiplier = val_list[y]
    for x in predefined_spacy:
      #print(x.similarity(ctm_spacy[y]))
      sim.append((x.similarity(ctm_spacy[y]) * 1.27 - 0.3) * 2 * multiplier)
    word_sum = np.sum(sim)
    word_sum /= val_total
    word_sum /= predefined_len
    total += word_sum

  topic_totals.append(total)

topic_totals

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


[-0.2125357643245855,
 0.11821742204429164,
 0.23898787709831312,
 -0.036740653376940484,
 0.060599357293911596,
 -0.05316673897716628,
 -0.1348498196345142,
 0.08583804043377856,
 0.1006703490260914,
 0.22041942408556006,
 0.2769698964617206,
 0.1767450370289452,
 0.24041487902846195,
 0.010684225636828713,
 0.14203802095446072,
 0.3449368470322408,
 -0.07945305517562853,
 0.0873330893257867,
 -0.16882107769666316,
 0.11532106429794478,
 0.2536777076490119,
 0.06567583766253857]

In [ ]:
p_topics[18]

['volcan', 'lava', 'tephra', 'tuff', 'pyroclastic']

a demonstration of finding the similarity between two word lists

In [ ]:
list1 = p_topics[18]
list2 = ctm_output['topics'][2]

list1SpacyDocs = [nlp(x) for x in list1]
list2SpacyDocs = [nlp(y) for y in list2]

similarityMatrix = [[x.similarity(y) for x in list1SpacyDocs] for y in list2SpacyDocs]
print(similarityMatrix)

[[0.9363662966996559, 0.7507102837664991, 0.6605866553813353, 0.5663370759165457, 0.6792918605346557], [0.9478613633586648, 0.7304177718000717, 0.5914136521398407, 0.4982682263304035, 0.5336252462068931], [0.21841191618875416, 0.08695412271719097, 0.08293284430840783, 0.01515912692049769, 0.3694243008887763], [0.615586099784941, 0.4208727737476735, 0.3755181933109901, 0.27451508992739865, 0.5647646860636913], [0.1370248911470287, 0.15204557205888367, 0.08596678974870754, 0.2639263275595089, 0.1934434924207559], [0.25446664813516073, 0.20145682031564108, 0.15710524864537548, 0.11552769164281344, 0.37453638673294554], [0.5502044657140456, 0.5678814865802275, 0.5488288135176702, 0.41128383146721115, 0.4726476193229746], [0.5095503638526483, 0.2807117336109552, 0.2579601412935704, 0.17393778009779778, 0.3731453560838655], [0.3645632201036115, 0.34900658371001997, 0.32423331308132847, 0.34764797697515276, 0.4773169755647167], [0.2327674011646529, 0.12751781265056586, 0.10128707494740989, 0.

In [ ]:
list1 = ['volcan']
list2 = ['volcano']

list1SpacyDocs = [nlp(x) for x in list1]
list2SpacyDocs = [nlp(x) for x in list2]

similarityMatrix = [[x.similarity(y) for x in list1SpacyDocs] for y in list2SpacyDocs]
print(similarityMatrix)

[[0.9478613633586648]]


In [ ]:
list1 = ['volcanic']
list2 = ['tephra']

list1SpacyDocs = [nlp(x) for x in list1]
list2SpacyDocs = [nlp(x) for x in list2]

similarityMatrix = [[x.similarity(y) for x in list1SpacyDocs] for y in list2SpacyDocs]
print(similarityMatrix)

[[0.6605866553813353]]


In [ ]:
for i in ctm_output['topics']:
  print(" ".join(i[0:6]))

summer diurnal monsoon cycle season convective
china product evaluation gpm imerg mainland
volcanic volcano emission eruption so large
trmm daily monthly b gauge gpm
solar radiation irradiance uv ultraviolet aura
vegetation forest change climate amazon dynamic
sub east omi asia tropospheric i
drought flood index case study sensing
span small style variant caps font
rainfall rain estimate trmm gauge datum
no level satellite moisture soil remote
africa south tropical southern cyclone america
rainfall satellite evaluation gauge iran estimate
learning machine neural microwave network error
radar precipitation rain global mission tropical
monsoon summer indian asian tropical convection
snow cover plateau tibetan snowfall mountain
high resolution rainfall heavy estimate west
atmospheric transport ocean convection sea eastern
arctic ice temperature cloud inversion winter
resolution high base estimate precipitation a
basin river hydrological upper evaluation dataset
aerosol satellite observati

Creating an empty dataframe with the columns (topics from our predefined list) we want

In [ ]:
s = ' Air_Quality Weather Climate Atmospheric_Chemistry Ocean_Biology Fires Agri/Aquaculture Vegetation Wildlife Severe_Weather Water_Resources Water_Quality Validation Algorithm Dust_Storm Floods Health Cyclones Volcanos Landslides Calibration Earthquakes'
c = s.split()
df = pd.DataFrame(columns=c)

We loop through our predefined_topics and our 40 CTM topics, and vectorize each list using spacy's nlp(x). We take the cosine similarity, and apply a simple linear function to separate values (see readme for more info). We sum over all combinations between a topic from each list to get the similarity between that pairing, and do this for all topics to generate a spreadsheet with similarity values.

In [ ]:
for t in range(len(p_topics)):

  topic_totals = []

  for i in range(40):

    predefined_len = len(p_topics[t])
    predefined_spacy = [nlp(x) for x in p_topics[t]]
    ctm_spacy = [nlp(x) for x in ctm_output['topics'][i]]
    val_list = vals[i]
    val_total = np.sum(val_list)

    total = 0
    for y in range(8):
      sim = []
      multiplier = val_list[y]
      for x in predefined_spacy:
        #print(x.similarity(ctm_spacy[y]))
        sim.append((x.similarity(ctm_spacy[y]) * 1.25 - 0.4) * 3 * multiplier)
      word_sum = np.sum(sim)
      word_sum /= val_total
      word_sum /= predefined_len
      total += word_sum

    topic_totals.append(total)
  
  df[c[t]] = topic_totals

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


In [ ]:
for t in range(len(p_topics)):

  topic_totals = []

  for i in range(40):

    predefined_len = len(p_topics[t])
    predefined_spacy = [nlp(x) for x in p_topics[t]]
    ctm_spacy = [nlp(x) for x in ctm_output['topics'][i]]
    val_list = vals[i]
    val_total = np.sum(val_list)

    total = 0
    for y in range(3):
      sim = []
      multiplier = val_list[y]
      for x in predefined_spacy:
        #print(x.similarity(ctm_spacy[y]))
        sim.append((x.similarity(ctm_spacy[y]) * 1.3 - 0.4) * 3 * multiplier)
      word_sum = np.sum(sim)
      word_sum /= val_total
      word_sum /= predefined_len
      total += word_sum

    topic_totals.append(total)
  
  df[c[t]] = topic_totals

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


Renaming indices for final dataframe, where indices are the generated topic names

In [ ]:
for i in range(len(ctm_output['topics'])):
  row_name = (" ".join(ctm_output['topics'][i][0:6]))
  df = df.rename(index={i:row_name})

Saving csv

In [ ]:
df.to_csv('/content/drive/My Drive/nasa/OCTIS/link_table.csv')

Removing nan

In [ ]:
df[df < 0] = np.nan

In [ ]:
df

,Air_Quality,Weather,Climate,Atmospheric_Chemistry,Ocean_Biology,Fires,Agri/Aquaculture,Vegetation,Wildlife,Severe_Weather,...,Validation,Algorithm,Dust_Storm,Floods,Health,Cyclones,Volcanos,Landslides,Calibration,Earthquakes
summer diurnal monsoon cycle season convective,-0.434339,0.021795,-0.012873,-0.360717,-0.117138,-0.280156,-0.321362,-0.198608,-0.162783,0.016065,...,-0.471594,-0.526625,0.036837,-0.118031,-0.354732,0.026294,-0.225726,-0.113497,-0.374544,-0.177740
china product evaluation gpm imerg mainland,-0.424425,-0.368516,-0.229158,-0.346788,-0.378876,-0.634284,-0.409895,-0.158102,-0.333533,-0.418877,...,0.034695,-0.370273,-0.396827,-0.332035,-0.384833,-0.564448,-0.436833,-0.574575,0.117033,-0.553573
volcanic volcano emission eruption so large,-0.130655,0.094624,0.127850,0.136056,0.127972,-0.012147,-0.313216,0.048733,-0.109004,0.122658,...,-0.134105,-0.257217,0.270178,0.036153,-0.162997,0.051451,0.423396,0.214016,-0.002754,0.170139
trmm daily monthly b gauge gpm,-0.526776,-0.423866,-0.394241,-0.454931,-0.444622,-0.587523,-0.464337,-0.481074,-0.427878,-0.466761,...,-0.352862,-0.448201,-0.541426,-0.413188,-0.502915,-0.519692,-0.598463,-0.538710,-0.332898,-0.525924
solar radiation irradiance uv ultraviolet aura,-0.161587,0.152930,0.180929,0.149533,0.078848,-0.191570,-0.328357,0.044219,-0.078373,0.129870,...,0.385770,-0.037264,-0.045749,0.129858,-0.035858,-0.165655,-0.208621,-0.120741,0.575979,-0.087301
vegetation forest change climate amazon dynamic,-0.187670,0.136748,0.292865,0.018712,0.190086,-0.033850,-0.088787,0.335928,0.285638,0.080250,...,0.264940,-0.070218,0.089953,0.272850,-0.204389,-0.106433,-0.071728,0.073745,0.261448,-0.128258
sub east omi asia tropospheric i,-0.523507,-0.404154,-0.371958,-0.450805,-0.333449,-0.558802,-0.487943,-0.363299,-0.488606,-0.358905,...,-0.277652,-0.358990,-0.360824,-0.332192,-0.505659,-0.316415,-0.414518,-0.415535,-0.218838,-0.492919
drought flood index case study sensing,-0.237770,0.099830,0.195561,-0.074710,-0.027870,0.002913,-0.144122,-0.013147,-0.005267,0.231313,...,-0.020053,-0.150385,0.196610,0.327940,-0.139737,0.187421,-0.177330,0.134687,0.013205,0.073195
span small style variant caps font,-0.613612,-0.332064,-0.345598,-0.556983,-0.317641,-0.560090,-0.535454,-0.256561,-0.332260,-0.249691,...,-0.408920,-0.351895,-0.359503,-0.392487,-0.549413,-0.495434,-0.438709,-0.388118,-0.148044,-0.467338
rainfall rain estimate trmm gauge datum,-0.227308,0.340700,0.118507,-0.033099,-0.034807,-0.038997,-0.278396,-0.053991,-0.183316,0.309690,...,-0.286143,-0.311825,0.374344,0.157811,-0.198708,0.256231,-0.046572,0.241739,-0.112536,0.069404


Saving csv with <0 removed

In [ ]:
df.to_csv('/content/drive/My Drive/nasa/OCTIS/link_table_na.csv')